In [3]:
import keras
#Handwritten Labelled Dataset
from keras.datasets import mnist
from keras.models import Sequential
#Dense Means fully connected layers , Dropout is a technique to improve convergence ,
# Flatten is to reshape the matrix for giving to different layers
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [4]:
#mini batch gradient descent ftw
batch_size = 128
#10 difference characters
num_classes = 10
#very short training time
epochs = 12

In [5]:
#input image dimensions
#28x28 pixel images. 
img_rows, img_cols = 28, 28

In [6]:
# the data downloaded, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
#this assumes our data format
#For 3D data, "channels_last" assumes (conv_dim1, conv_dim2, conv_dim3, channels) while 
#"channels_first" assumes (channels, conv_dim1, conv_dim2, conv_dim3).
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [8]:
#more reshaping
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

In [11]:
#again
model.add(Conv2D(128, (3, 3), activation='relu'))
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
#randomly turn neurons on and off to improve convergence
model.add(Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
model.add(Flatten())

In [13]:
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
model.add(Dropout(0.5))

In [14]:
#output a softmax to squash the matrix into output probabilities
model.add(Dense(num_classes, activation='softmax'))
#Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
#categorical ce since we have multiple classes (10) 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


In [15]:
#train that ish!
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 288s 5ms/step - loss: 0.2636 - acc: 0.9195 - val_loss: 0.0571 - val_acc: 0.9805
Epoch 2/12
60000/60000 [==============================] - 441s 7ms/step - loss: 0.0880 - acc: 0.9739 - val_loss: 0.0478 - val_acc: 0.9836
Epoch 3/12
60000/60000 [==============================] - 355s 6ms/step - loss: 0.0652 - acc: 0.9797 - val_loss: 0.0362 - val_acc: 0.9871
Epoch 4/12
60000/60000 [==============================] - 230s 4ms/step - loss: 0.0556 - acc: 0.9837 - val_loss: 0.0360 - val_acc: 0.9873
Epoch 5/12
60000/60000 [==============================] - 221s 4ms/step - loss: 0.0462 - acc: 0.9862 - val_loss: 0.0317 - val_acc: 0.9897
Epoch 6/12
60000/60000 [==============================] - 225s 4ms/step - loss: 0.0423 - acc: 0.9874 - val_loss: 0.0317 - val_acc: 0.9897
Epoch 7/12
60000/60000 [==============================] - 222s 4ms/step - loss: 0.0386 - acc: 0.9884 - val_loss: 0.0296 - 

In [18]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
